In [1]:
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.express as px
from itertools import product

SELECT id_municipio, 
peso_amostral, 
v0636 AS onde_estudava, 
v6364 AS cod_mun_estudava, 
V0660 AS onde_Trabalhava,
V6604 AS cod_mun_trabalho, 
V0661 AS retorno_diario, 
V0662 AS tempo_deslocamento
FROM basedosdados.br_ibge_censo_demografico.microdados_pessoa_2010

In [2]:
df_arranjos = pd.read_csv('data/pop/arranjos populacionais/tab01.csv', sep=';', thousands='.', decimal=',')

In [3]:
lista_municipios = tuple(df_arranjos['Código do município'].values)

In [4]:
df_arranjos

,Arranjo,municipio,CodArranjo,Código do município,População,Núcleo,Pessoas que trabalham e estudam em outros municípios do arranjo,Índice de integração do município com o arranjo,Mancha Urbanizada Contígua,"Município desmembrado de outro do arranjo, após 1988",Nível,Via,Iteração
0,Adamantina - Lucélia/SP,Adamantina,3500105,3500105,33797,Sim,2268,0.10,Sim,Não,1,-,0
1,Adamantina - Lucélia/SP,Inúbia Paulista,3500105,3520806,363,Não,423,0.17,Não,Não,1,-,1
2,Adamantina - Lucélia/SP,Lucélia,3500105,3527405,19882,Sim,2289,0.18,Sim,Não,1,-,0
3,Adamantina - Lucélia/SP,Mariápolis,3500105,3528908,3916,Não,499,0.20,Não,Não,1,-,1
4,Adamantina - Lucélia/SP,Pracinha,3500105,3540853,2858,Não,261,0.26,Não,Sim,1,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Volta Redonda - Barra Mansa/RJ,Volta Redonda,3306305,3306305,257803,Sim,24057,0.14,Sim,Não,1,-,0
949,Votuporanga/SP,Álvares Florence,3557105,3501202,3897,Não,609,0.25,Não,Não,1,-,0
950,Votuporanga/SP,Parisi,3557105,3536257,2032,Não,464,0.32,Não,Sim,1,-,0
951,Votuporanga/SP,Sebastianópolis do Sul,3557105,3551306,3031,Não,797,0.38,Não,Não,1,-,0


In [5]:
df_arranjos['Destino'] = df_arranjos['Código do município']

In [6]:
df_arranjos.rename(columns={'Código do município':'Cod_Mun_Origem', 'municipio':'ArranjoOrigem', 'CodArranjo':'Cod_Arranjo_Origem'}, inplace=True)

In [7]:
df_arranjos['Cod_Arranjo_Destino'] = df_arranjos.Destino.replace({key:value for key, value in zip(df_arranjos['Cod_Mun_Origem'].values, df_arranjos['Cod_Arranjo_Origem'].values)})

In [8]:
df_arranjos['Arranjo_Destino']  = df_arranjos.Destino.replace({key:value for key, value in zip(df_arranjos['Cod_Mun_Origem'].values, df_arranjos['ArranjoOrigem'].values)})

In [9]:
df_arranjos_tratado = df_arranjos[['Arranjo', 'ArranjoOrigem', 'Cod_Arranjo_Origem', 'Cod_Mun_Origem', 'Destino', 'Cod_Arranjo_Destino', 'Arranjo_Destino']]

In [14]:
df_arranjos_tratado_filtrado = df_arranjos_tratado[df_arranjos_tratado['Cod_Arranjo_Origem'] == 4106902]

In [15]:
df_arranjos_tratado_filtrado_pivot = df_arranjos_tratado_filtrado.pivot_table(index=['Cod_Mun_Origem'], columns=['Destino'], values=['Cod_Arranjo_Destino'])

In [16]:
df_arranjos_tratado_filtrado_pivot.droplevel(0, axis=1).columns

Int64Index([4100400, 4101804, 4102307, 4103107, 4104006, 4104204, 4104253,
            4105805, 4106209, 4106902, 4107652, 4111258, 4114302, 4119152,
            4119509, 4120804, 4122206, 4125506],
           dtype='int64', name='Destino')

In [17]:
list_origin = []
list_destination = []

In [18]:

for origin, destination in product(df_arranjos_tratado_filtrado.Cod_Mun_Origem, df_arranjos_tratado_filtrado.Cod_Mun_Origem):
    list_origin.append(origin)
    list_destination.append(destination)

In [19]:
df_cwb = pd.DataFrame(data={'cod_mun_origin':list_origin, 'cod_mun_destination':list_destination, 'ArranjoPopulacional':'Curitiba (PR)', 'CodArranjoPopulacional':4106902})

In [20]:
df = pd.read_feather('data/pop/microdados_amostra/microdata_2010_census.ftd')

In [21]:
df_cwb['cod_mun_origin'].unique()

array([4100400, 4101804, 4102307, 4103107, 4104006, 4104204, 4104253,
       4105805, 4106209, 4106902, 4107652, 4111258, 4114302, 4119152,
       4119509, 4120804, 4122206, 4125506], dtype=int64)

In [22]:
filter_origin = df.id_municipio.isin(df_cwb['cod_mun_origin'].unique())
filter_destination_education = df.cod_mun_estudava.isin(df_cwb['cod_mun_origin'].unique())
filter_destination_work = df.cod_mun_trabalho.isin(df_cwb['cod_mun_origin'].unique())

In [36]:
#df[(filter_origin)].to_clipboard()#.groupby(by=['id_municipio', 'cod_mun_estudava'], group_keys=False).sum()[['peso_amostral']].reset_index()
#df_commuting_for_studies = df[(filter_origin) & (filter_destination_education)].info()
#df_commuting_for_studies.to_clipboard()
#df[(filter_origin) & (filter_destination_work)].info()
#df_commuting_for_work.to_clipboard()
#df_commuting_for_studies_and_work.to_clipboard()
df_commuting_for_studies_and_work.to_clipboard()

In [35]:
df_commuting_for_studies = df[(filter_origin) & (filter_destination_education)].groupby(by=['id_municipio', 'cod_mun_estudava'], group_keys=False).sum()[['peso_amostral']].reset_index()

In [25]:
df_commuting_for_work = df[(filter_origin) & (filter_destination_work)].groupby(by=['id_municipio', 'cod_mun_trabalho'], group_keys=False).sum()[['peso_amostral']].reset_index()

## PROVAVELMENTE TEM UM PROBLEMA NO FILTRO ABAIXO. tALVEZ EU TENHA QUE FAZER PARA UMA FRAÇÃO MENOR, PARA PODER VER O QUE OCORRE. A DIFERENÇA É POUCA, MAS OCORRE QUE APARECEM CASOS DE HAVER PESSOAS QUE TRABALHAM E ESTUDAM FORA, SEM QUE HAJA PESSOAS QUE TRABALHAM FORA, OU SEM QUE HAJA PESSOAS QUE ESTUDAM FORA. UMA BOA FORMA DE VERIFICAR ISSO É TIRAR O GROUPBY.SUM() E VER OS VALORES EFETIVAMENTE LISTADOS PARA VÁRIOS MUNICÍPIOS, PARA IDENTIFICAR O QUE ESTÁ DANDO ERRADO.

In [33]:
df_commuting_for_studies_and_work = df[(filter_origin) & (filter_destination_education) & (filter_destination_work)].groupby(by=['id_municipio', 'cod_mun_estudava', 'cod_mun_trabalho'], group_keys=False).sum()[['peso_amostral']].reset_index()

In [94]:
df_commuting_for_studies[df_commuting_for_studies['id_municipio'] == 4125506]

,id_municipio,cod_mun_estudava,peso_amostral
110,4125506,4100400.0,1
111,4125506,4101804.0,4
112,4125506,4105805.0,1
113,4125506,4106902.0,875
114,4125506,4107652.0,1
115,4125506,4114302.0,1
116,4125506,4119152.0,3
117,4125506,4119509.0,3
118,4125506,4120804.0,1


In [95]:
df_commuting_for_work[df_commuting_for_work['id_municipio'] == 4125506]

,id_municipio,cod_mun_trabalho,peso_amostral
179,4125506,4100400.0,5
180,4125506,4101804.0,47
181,4125506,4104006.0,5
182,4125506,4104204.0,8
183,4125506,4104253.0,1
184,4125506,4105805.0,28
185,4125506,4106902.0,2701
186,4125506,4107652.0,13
187,4125506,4119152.0,88
188,4125506,4119509.0,32


In [96]:
df_commuting_for_studies_and_work[df_commuting_for_studies_and_work['id_municipio'] == 4125506]

,id_municipio,cod_mun_estudava,cod_mun_trabalho,peso_amostral
143,4125506,4101804.0,4101804.0,1
144,4125506,4101804.0,4119152.0,1
145,4125506,4106902.0,4101804.0,6
146,4125506,4106902.0,4104006.0,1
147,4125506,4106902.0,4105805.0,1
148,4125506,4106902.0,4106902.0,233
149,4125506,4106902.0,4119152.0,6
150,4125506,4106902.0,4119509.0,1
151,4125506,4106902.0,4120804.0,2
152,4125506,4107652.0,4107652.0,1


In [151]:
df_commuting_for_studies_and_work.rename(columns={'cod_mun_trabalho':'cod_mun_trabalho_estudo'}, inplace=True)

In [63]:
df_commuting_for_studies_and_work

,id_municipio,cod_mun_estudava,cod_mun_trabalho,peso_amostral
0,4107900,4111100.0,4115200.0,2
1,4107900,4115200.0,4111100.0,1
2,4107900,4115200.0,4115200.0,23
3,4110003,4115200.0,4115200.0,10
4,4111100,4107900.0,4115200.0,1
5,4111100,4114807.0,4115200.0,1
6,4111100,4115200.0,4115200.0,9
7,4114104,4115200.0,4110003.0,1
8,4114104,4115200.0,4115200.0,24
9,4114104,4115200.0,4126256.0,1


In [60]:
df_commuting_for_studies_and_work[df_commuting_for_studies_and_work['id_municipio'] == 4126256]

,id_municipio,cod_mun_estudava,cod_mun_trabalho,peso_amostral
21,4126256,4111100.0,4115200.0,1
22,4126256,4114807.0,4115200.0,1
23,4126256,4115200.0,4114807.0,1
24,4126256,4115200.0,4115200.0,66
25,4126256,4117503.0,4115200.0,1


In [30]:
df_commuting_for_work_or_studies = pd.merge(left=df_commuting_for_studies, right=df_commuting_for_work
         , left_on=['id_municipio', 'cod_mun_estudava']
         , right_on=['id_municipio', 'cod_mun_trabalho']
         , how='outer'
        )

In [31]:
df_commuting_for_studies_and_work = pd.merge(left=df_commuting_for_work_or_studies, right=df_commuting_for_studies_and_work
         , left_on=['id_municipio', 'cod_mun_estudava', 'cod_mun_trabalho']
         , right_on=['id_municipio', 'cod_mun_estudava', 'cod_mun_trabalho']
         , how='outer'
        )

In [32]:
df_commuting_for_studies_and_work.to_clipboard()

In [107]:
df_commuting_for_studies_and_work[df_commuting_for_studies_and_work['id_municipio'] == 4125506]

,id_municipio,cod_mun_estudava,peso_amostral_x,cod_mun_trabalho,peso_amostral_y,peso_amostral
110,4125506,4100400.0,9.292659,4100400.0,55.620529,NaN
111,4125506,4101804.0,42.868518,4101804.0,481.117076,11.412838
112,4125506,4105805.0,11.314225,4105805.0,281.924614,NaN
113,4125506,4106902.0,8945.622217,4106902.0,27888.593425,2399.482797
114,4125506,4107652.0,10.980068,4107652.0,126.092146,10.980068
115,4125506,4114302.0,3.206236,NaN,NaN,NaN
116,4125506,4119152.0,31.026396,4119152.0,874.232877,NaN
117,4125506,4119509.0,28.981155,4119509.0,313.070967,8.673949
118,4125506,4120804.0,7.341571,4120804.0,125.871139,7.341571
192,4125506,NaN,NaN,4104006.0,58.827950,NaN


In [146]:
df_commuting_for_work_or_studies[df_commuting_for_work_or_studies['id_municipio'] == 4119152]

,id_municipio,cod_mun_estudava,peso_amostral_x,cod_mun_trabalho,peso_amostral_y
83,4119152,4100400.0,10.769652,4100400.0,53.521376
84,4119152,4104006.0,18.317651,4104006.0,173.770183
85,4119152,4104204.0,11.320957,4104204.0,21.174201
86,4119152,4105805.0,213.846903,4105805.0,1106.881987
87,4119152,4106902.0,7944.428100,4106902.0,22659.776942
88,4119152,4119509.0,260.955605,4119509.0,636.535081
89,4119152,4120804.0,30.013757,4120804.0,216.133626
90,4119152,4125506.0,53.155949,4125506.0,1352.538319
176,4119152,NaN,NaN,4101804.0,306.364671
177,4119152,NaN,NaN,4104253.0,12.726571


In [126]:
df_final = pd.DataFrame(columns=)

In [127]:
df_final.append(df_commuting_for_work_or_studies)

,id_municipio,cod_mun_estudava,peso_amostral_x,cod_mun_trabalho,peso_amostral_y
0,4100400,4103107.0,12.659084,NaN,NaN
1,4100400,4104253.0,28.757276,4104253.0,62.150029
2,4100400,4105805.0,356.487072,4105805.0,683.297022
3,4100400,4106902.0,7682.828847,4106902.0,28416.484770
4,4100400,4107652.0,16.725843,4107652.0,7.009160
...,...,...,...,...,...
190,4122206,NaN,NaN,4107652.0,14.924314
191,4122206,NaN,NaN,4125506.0,31.419551
192,4125506,NaN,NaN,4104006.0,58.827950
193,4125506,NaN,NaN,4104204.0,78.986828


In [116]:
df_commuting_for_work

peso_amostral
id_municipio cod_mun_trabalho               
4100400      4101804.0            179.576621
             4104006.0             46.710427
             4104204.0            106.876143
             4104253.0             62.150029
             4105805.0            683.297022
...                                      ...
4125506      4106902.0          27888.593425
             4107652.0            126.092146
             4119152.0            874.232877
             4119509.0            313.070967
             4120804.0            125.871139

[190 rows x 1 columns]

In [78]:
df_microdados_pr[df_microdados_pr.id_municipio.isin(df_cwb['cod_mun_origin'].unique())]

,id_municipio,peso_amostral,frequentava_escola,onde_estudava,cod_mun_estudava,onde_Trabalhava,cod_mun_trabalho,retorno_diario,tempo_deslocamento
1149,4100400,5.929634,3,NaN,NaN,4.0,NaN,2.0,NaN
1150,4101804,10.624679,3,NaN,NaN,4.0,NaN,2.0,NaN
1162,4104204,12.276617,3,NaN,NaN,4.0,NaN,2.0,NaN
1172,4105805,9.362806,3,NaN,NaN,4.0,NaN,2.0,NaN
1173,4105805,4.540818,3,NaN,NaN,4.0,NaN,2.0,NaN
...,...,...,...,...,...,...,...,...,...
8325978,4125506,10.351727,3,NaN,NaN,5.0,NaN,NaN,NaN
8325979,4125506,10.738170,3,NaN,NaN,5.0,NaN,NaN,NaN
8325980,4125506,9.892686,3,NaN,NaN,5.0,NaN,NaN,NaN
8325981,4125506,10.645375,3,NaN,NaN,5.0,NaN,NaN,NaN


In [74]:
list_destination

[3500105,
 3520806,
 3527405,
 3528908,
 3540853,
 3500105,
 3520806,
 3527405,
 3528908,
 3540853,
 3500105,
 3520806,
 3527405,
 3528908,
 3540853,
 3500105,
 3520806,
 3527405,
 3528908,
 3540853,
 3500105,
 3520806,
 3527405,
 3528908,
 3540853]

In [69]:
{origin:destination for origin, destination in product(df_arranjos_tratado_filtrado.Cod_Mun_Origem, df_arranjos_tratado_filtrado.Cod_Mun_Origem)}

{3500105: 3540853,
 3520806: 3540853,
 3527405: 3540853,
 3528908: 3540853,
 3540853: 3540853}

In [61]:
df_arranjos['Destino'] = df_arranjos['Código do município']

In [61]:
df_arranjos['Destino'] = df_arranjos['Código do município']

In [61]:
df_arranjos_tratado_filtrado_pivot.droplevel(0, axis=1).melt(id_vars=df_arranjos_tratado_filtrado_pivot.droplevel(0, axis=1).columns, )

,3500105,3520806,3527405,3528908,3540853,Destino,value


In [30]:
df_arranjos_tratado.groupby(['Cod_Mun_Origem', 'Destino']).count()

,,Arranjo,ArranjoOrigem,Cod_Arranjo_Origem,Cod_Arranjo_Destino,Arranjo_Destino
Cod_Mun_Origem,Destino,,,,,
1100106,1100106,1,1,1,1,1
1100205,1100205,1,1,1,1,1
1100809,1100809,1,1,1,1,1
1200054,1200054,1,1,1,1,1
1200104,1200104,1,1,1,1,1
...,...,...,...,...,...,...
5221403,5221403,1,1,1,1,1
5221551,5221551,1,1,1,1,1
5221809,5221809,1,1,1,1,1


In [32]:
df_arranjos_pivot =  df_arranjos_tratado.pivot(index=['Cod_Mun_Origem'], columns=['Destino'], values='Arranjo')

In [38]:
columns = list(df_arranjos_pivot.columns)

In [41]:
df_arranjos_pivot.reset_index().melt(columns.append('Cod_Mun_Origem'))

,Destino,value
0,Cod_Mun_Origem,1100106
1,Cod_Mun_Origem,1100205
2,Cod_Mun_Origem,1100809
3,Cod_Mun_Origem,1200054
4,Cod_Mun_Origem,1200104
...,...,...
909157,5300108,NaN
909158,5300108,NaN
909159,5300108,NaN
909160,5300108,NaN


In [26]:
df = bd.read_sql(f'SELECT id_municipio, peso_amostral, V0628 AS frequentava_escola, v0636 AS onde_estudava, v6364 AS cod_mun_estudava, V0660 AS onde_Trabalhava, V6604 AS cod_mun_trabalho, V0661 AS retorno_diario, V0662 AS tempo_deslocamento FROM basedosdados.br_ibge_censo_demografico.microdados_pessoa_2010 WHERE id_municipio in {lista_municipios}', billing_project_id='urbanresearch-305315')

Downloading: 100%|████████████████████████████████████████████████████████████████████| 8348734/8348734 [11:23<00:00, 12215.08rows/s]


In [29]:
df.to_feather('data/pop/microdados_amostra/microdata_2010_census.ftd')

In [33]:
df.columns

Index(['id_municipio', 'peso_amostral', 'frequentava_escola', 'onde_estudava',
       'cod_mun_estudava', 'onde_Trabalhava', 'cod_mun_trabalho',
       'retorno_diario', 'tempo_deslocamento'],
      dtype='object')

In [45]:
df[df.frequentava_escola.isin((1,2))][df.id_municipio == 4126256][df.cod_mun_estudava == 4115200].peso_amostral.sum()

2993.358157288821

In [47]:
df[df.frequentava_escola.isin((1,2))][df.id_municipio == 4126256][df.cod_mun_estudava == 4115200].id_municipio.count()

285

In [68]:
df_arranjos.pivot(index=['Código do município'], columns=['Destino'], values='Arranjo')

Destino,1100106,1100205,1100809,1200054,1200104,1200252,1303007,1304062,1400159,1500503,...,5219738,5219753,5220454,5220686,5221304,5221403,5221551,5221809,5221858,5300108
Código do município,,,,,,,,,,,,,,,,,,,,,
1100106,Internacional de Guajará-Mirim/Brasil - Guayar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1100205,NaN,Porto Velho/RO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1100809,NaN,NaN,Porto Velho/RO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1200054,NaN,NaN,NaN,Internacional de Assis Brasil/Brasil,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1200104,NaN,NaN,NaN,NaN,Internacional de Cobija/Bolívia - Brasiléia/Br...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5221403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Goiânia/GO,NaN,NaN,NaN,NaN
5221551,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Maurilândia/GO,NaN,NaN,NaN
5221809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pires do Rio/GO,NaN,NaN


In [61]:
df_arranjos['Destino'] = df_arranjos['Código do município']

In [72]:
teste_adamantina = df_arranjos[['Código do município', 'Destino', 'Arranjo']][df_arranjos.CodArranjo == 3500105].pivot(index=['Código do município'], columns=['Destino'], values='Arranjo')

In [80]:
teste_adamantina

Destino,3500105,3520806,3527405,3528908,3540853
Código do município,,,,,
3500105,Adamantina - Lucélia/SP,NaN,NaN,NaN,NaN
3520806,NaN,Adamantina - Lucélia/SP,NaN,NaN,NaN
3527405,NaN,NaN,Adamantina - Lucélia/SP,NaN,NaN
3528908,NaN,NaN,NaN,Adamantina - Lucélia/SP,NaN
3540853,NaN,NaN,NaN,NaN,Adamantina - Lucélia/SP


In [91]:
['Código do Município'].extend(teste_adamantina.columns[:])

In [99]:
listacod = ['Código do Município']

In [95]:
list(teste_adamantina.columns)[:]

[3500105, 3520806, 3527405, 3528908, 3540853]

In [100]:
listacod.append(list(teste_adamantina.columns)[:])

In [101]:
listacod

['Código do Município', [3500105, 3520806, 3527405, 3528908, 3540853]]

In [98]:
x

In [90]:
teste_adamantina.reset_index().melt(teste_adamantina.columns[:])

Destino,Código do município,3500105,3520806,3527405,3528908,3540853
0,3500105,Adamantina - Lucélia/SP,NaN,NaN,NaN,NaN
1,3520806,NaN,Adamantina - Lucélia/SP,NaN,NaN,NaN
2,3527405,NaN,NaN,Adamantina - Lucélia/SP,NaN,NaN
3,3528908,NaN,NaN,NaN,Adamantina - Lucélia/SP,NaN
4,3540853,NaN,NaN,NaN,NaN,Adamantina - Lucélia/SP


In [78]:
df_arranjos[df_arranjos.CodArranjo == 3500105].groupby(['Código do município', 'Código do município']).count()

,,Arranjo,Arranjos populacionais e municípios brasileiros,CodArranjo,População,Núcleo,Pessoas que trabalham e estudam em outros municípios do arranjo,Índice de integração do município com o arranjo,Mancha Urbanizada Contígua,"Município desmembrado de outro do arranjo, após 1988",Nível,Via,Iteração,Destino
Código do município,Código do município,,,,,,,,,,,,,
3500105,3500105,1,1,1,1,1,1,1,1,1,1,1,1,1
3520806,3520806,1,1,1,1,1,1,1,1,1,1,1,1,1
3527405,3527405,1,1,1,1,1,1,1,1,1,1,1,1,1
3528908,3528908,1,1,1,1,1,1,1,1,1,1,1,1,1
3540853,3540853,1,1,1,1,1,1,1,1,1,1,1,1,1


In [52]:
df_arranjos.groupby()

TypeError: You have to supply one of 'by' and 'level'

## Tentativa com abordagem programática

tratar com teorias de conjuntos e permutação passou a ter muitos problemas. Por essa razão, 

In [37]:
df_arranjos = pd.read_csv('data/pop/arranjos populacionais/tab01.csv', sep=';', thousands='.', decimal=',')

In [38]:
lista_municipios = tuple(df_arranjos['Código do município'].values)

In [39]:
df_arranjos

,Arranjo,municipio,CodArranjo,Código do município,População,Núcleo,Pessoas que trabalham e estudam em outros municípios do arranjo,Índice de integração do município com o arranjo,Mancha Urbanizada Contígua,"Município desmembrado de outro do arranjo, após 1988",Nível,Via,Iteração
0,Adamantina - Lucélia/SP,Adamantina,3500105,3500105,33797,Sim,2268,0.10,Sim,Não,1,-,0
1,Adamantina - Lucélia/SP,Inúbia Paulista,3500105,3520806,363,Não,423,0.17,Não,Não,1,-,1
2,Adamantina - Lucélia/SP,Lucélia,3500105,3527405,19882,Sim,2289,0.18,Sim,Não,1,-,0
3,Adamantina - Lucélia/SP,Mariápolis,3500105,3528908,3916,Não,499,0.20,Não,Não,1,-,1
4,Adamantina - Lucélia/SP,Pracinha,3500105,3540853,2858,Não,261,0.26,Não,Sim,1,-,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Volta Redonda - Barra Mansa/RJ,Volta Redonda,3306305,3306305,257803,Sim,24057,0.14,Sim,Não,1,-,0
949,Votuporanga/SP,Álvares Florence,3557105,3501202,3897,Não,609,0.25,Não,Não,1,-,0
950,Votuporanga/SP,Parisi,3557105,3536257,2032,Não,464,0.32,Não,Sim,1,-,0
951,Votuporanga/SP,Sebastianópolis do Sul,3557105,3551306,3031,Não,797,0.38,Não,Não,1,-,0


In [40]:
df_arranjos['Destino'] = df_arranjos['Código do município']

In [41]:
df_arranjos.rename(columns={'Código do município':'Cod_Mun_Origem', 'municipio':'ArranjoOrigem', 'CodArranjo':'Cod_Arranjo_Origem'}, inplace=True)

In [42]:
df_arranjos['Cod_Arranjo_Destino'] = df_arranjos.Destino.replace({key:value for key, value in zip(df_arranjos['Cod_Mun_Origem'].values, df_arranjos['Cod_Arranjo_Origem'].values)})

In [43]:
df_arranjos['Arranjo_Destino']  = df_arranjos.Destino.replace({key:value for key, value in zip(df_arranjos['Cod_Mun_Origem'].values, df_arranjos['ArranjoOrigem'].values)})

In [44]:
df_arranjos_tratado = df_arranjos[['Arranjo', 'ArranjoOrigem', 'Cod_Arranjo_Origem', 'Cod_Mun_Origem', 'Destino', 'Cod_Arranjo_Destino', 'Arranjo_Destino']]

In [45]:
df_arranjos_tratado_filtrado = df_arranjos_tratado[df_arranjos_tratado['Cod_Arranjo_Origem'] == 4106902]

In [113]:
df_arranjos_tratado_filtrado.Cod_Mun_Origem.unique()

array([4100400, 4101804, 4102307, 4103107, 4104006, 4104204, 4104253,
       4105805, 4106209, 4106902, 4107652, 4111258, 4114302, 4119152,
       4119509, 4120804, 4122206, 4125506], dtype=int64)

In [136]:
df = pd.read_feather('data/pop/microdados_amostra/microdata_2010_census.ftd')

In [137]:
filter_origin = df.id_municipio.isin(df_arranjos_tratado_filtrado['Cod_Mun_Origem'].unique())
filter_not_na = ~((df_microdata_arranjo.cod_mun_estudava.isna()) & (df_microdata_arranjo.cod_mun_trabalho.isna()))
#df['cod_mun_estudava'] != df['cod_mun_trabalho']
filter_destination_education = df.cod_mun_estudava.isin(df_arranjos_tratado_filtrado['Cod_Mun_Origem'].unique())
filter_destination_work = df.cod_mun_trabalho.isin(df_arranjos_tratado_filtrado['Cod_Mun_Origem'].unique())

In [138]:
df_microdata_arranjo = df[filter_origin & filter_not_na]

In [139]:
df_microdata_arranjo = df_microdata_arranjo[~((df_microdata_arranjo.cod_mun_estudava.isna()) & (df_microdata_arranjo.cod_mun_trabalho.isna()))]

In [141]:
df_microdata_arranjo_movimento_heterogeneo = df_microdata_arranjo[(~df_microdata_arranjo['cod_mun_estudava'].isna()) & (~df_microdata_arranjo['cod_mun_trabalho'].isna())]

In [148]:
df_microdata_arranjo_movimento_heterogeneo = df_microdata_arranjo_movimento_heterogeneo[df_microdata_arranjo_movimento_heterogeneo.cod_mun_estudava != df_microdata_arranjo_movimento_heterogeneo.cod_mun_trabalho]

In [150]:
df_microdata_arranjo_movimento_heterogeneo.peso_amostral.sum()

4710.228824175836

In [157]:
df_arranjos['Arranjo'] == arranjo

,Arranjo,ArranjoOrigem,Cod_Arranjo_Origem,Cod_Mun_Origem,População,Núcleo,Pessoas que trabalham e estudam em outros municípios do arranjo,Índice de integração do município com o arranjo,Mancha Urbanizada Contígua,"Município desmembrado de outro do arranjo, após 1988",Nível,Via,Iteração,Destino,Cod_Arranjo_Destino,Arranjo_Destino
0,Adamantina - Lucélia/SP,Adamantina,3500105,3500105,33797,Sim,2268,0.10,Sim,Não,1,-,0,3500105,3500105,Adamantina
1,Adamantina - Lucélia/SP,Inúbia Paulista,3500105,3520806,363,Não,423,0.17,Não,Não,1,-,1,3520806,3500105,Inúbia Paulista
2,Adamantina - Lucélia/SP,Lucélia,3500105,3527405,19882,Sim,2289,0.18,Sim,Não,1,-,0,3527405,3500105,Lucélia
3,Adamantina - Lucélia/SP,Mariápolis,3500105,3528908,3916,Não,499,0.20,Não,Não,1,-,1,3528908,3500105,Mariápolis
4,Adamantina - Lucélia/SP,Pracinha,3500105,3540853,2858,Não,261,0.26,Não,Sim,1,-,0,3540853,3500105,Pracinha
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Volta Redonda - Barra Mansa/RJ,Volta Redonda,3306305,3306305,257803,Sim,24057,0.14,Sim,Não,1,-,0,3306305,3306305,Volta Redonda
949,Votuporanga/SP,Álvares Florence,3557105,3501202,3897,Não,609,0.25,Não,Não,1,-,0,3501202,3557105,Álvares Florence
950,Votuporanga/SP,Parisi,3557105,3536257,2032,Não,464,0.32,Não,Sim,1,-,0,3536257,3557105,Parisi
951,Votuporanga/SP,Sebastianópolis do Sul,3557105,3551306,3031,Não,797,0.38,Não,Não,1,-,0,3551306,3557105,Sebastianópolis do Sul


# Verificação do volume de casos de movimentos bipartidos

In [10]:
df = pd.read_feather('data/pop/microdados_amostra/microdata_2010_census.ftd')

In [11]:
list_movimentos_bipartidos = []
list_dicts = []

In [85]:
list_movimentos_bipartidos = []
for arranjo in df_arranjos['Arranjo'].unique():
#for arranjo in ['Maringá/PR', 'Curitiba/PR']:
    df_arranjos_filtrado = df_arranjos[df_arranjos['Arranjo'] == arranjo]
    
    filter_origin = df.id_municipio.isin(df_arranjos_filtrado['Cod_Mun_Origem'].unique())
    df_microdata_arranjo = df[filter_origin]
    
    filter_not_na = ~((df_microdata_arranjo.cod_mun_estudava.isna()) & (df_microdata_arranjo.cod_mun_trabalho.isna()))
    df_microdata_arranjo[filter_not_na]

    filter_not_na_complete = (~df_microdata_arranjo['cod_mun_estudava'].isna()) & (~df_microdata_arranjo['cod_mun_trabalho'].isna())
    df_microdata_arranjo_movimento_heterogeneo = df_microdata_arranjo[filter_not_na_complete]
    filter_heterogeneous_movement = df_microdata_arranjo_movimento_heterogeneo.cod_mun_estudava != df_microdata_arranjo_movimento_heterogeneo.cod_mun_trabalho
    df_microdata_arranjo_movimento_heterogeneo = df_microdata_arranjo_movimento_heterogeneo[filter_heterogeneous_movement]
    list_movimentos_bipartidos.append(round(df_microdata_arranjo_movimento_heterogeneo.peso_amostral.sum()))

    df_movimentos_homogeneos = df_microdata_arranjo[~df_microdata_arranjo.index.isin(df_microdata_arranjo_movimento_heterogeneo.index)]

    df_movimentos_homogeneos = df_movimentos_homogeneos[(~df_movimentos_homogeneos['cod_mun_estudava'].isna()) | (~df_movimentos_homogeneos['cod_mun_trabalho'].isna())]

    df_movimentos_homogeneos['tipoDeslocamento'] = ''

    if (df_movimentos_homogeneos.loc[:,'cod_mun_estudava'] == df_movimentos_homogeneos.loc[:,'cod_mun_trabalho']).sum() > 0:
        df_movimentos_homogeneos[df_movimentos_homogeneos.loc[:,'cod_mun_estudava'] == df_movimentos_homogeneos.loc[:,'cod_mun_trabalho']]['tipoDeslocamento'] = 'Trabalho e Estudo'

    df_movimentos_homogeneos.loc[df_movimentos_homogeneos.loc[:,'cod_mun_estudava'] == df_movimentos_homogeneos.loc[:,'cod_mun_trabalho']].at[:,'tipoDeslocamento'] = 'Trabalho e Estudo'

    index = df_movimentos_homogeneos.loc[df_movimentos_homogeneos.loc[:,'cod_mun_estudava'] == df_movimentos_homogeneos.loc[:,'cod_mun_trabalho']].index

    df_movimentos_homogeneos.loc[index, ['tipoDeslocamento']] = 'Trabalho e Estudo'

    index = df_movimentos_homogeneos[df_movimentos_homogeneos['tipoDeslocamento'] != 'Trabalho e Estudo'][~(df_movimentos_homogeneos['cod_mun_estudava'].isna())].index

    df_movimentos_homogeneos.loc[index, ['tipoDeslocamento']] = 'Estudo'

    index = df_movimentos_homogeneos[df_movimentos_homogeneos['tipoDeslocamento'] != 'Trabalho e Estudo'][~(df_movimentos_homogeneos['cod_mun_trabalho'].isna())].index

    df_movimentos_homogeneos.loc[index, ['tipoDeslocamento']] = 'Trabalho'

    df_movimentos_homogeneos['Arranjo'] = arranjo

    destinations = [int(row[1]['cod_mun_estudava']) if pd.notnull(row[1]['cod_mun_estudava']) else int(row[1]['cod_mun_trabalho']) for row in df_movimentos_homogeneos.iterrows()]

    df_movimentos_homogeneos['Destino'] = destinations
    
    df_grouped = df_movimentos_homogeneos.groupby(['Arranjo', 'tipoDeslocamento','id_municipio', 'Destino']).sum()['peso_amostral'].reset_index()

    df_grouped = df_grouped.pivot(index=['Arranjo', 'id_municipio', 'Destino'], columns=['tipoDeslocamento'])

    df_grouped = df_grouped.reset_index().fillna(0)
    
    if df_grouped.columns[-1][-1] != 'Trabalho e Estudo':

        df_grouped.columns = ['Arranjo', 'Origem', 'Destino', 'Estudo', 'Trabalho']

        df_grouped['Trabalho e Estudo'] = 0

    else:
        df_grouped.columns = ['Arranjo', 'Origem', 'Destino', 'Estudo', 'Trabalho', 'Trabalho e Estudo']

    df_grouped = df_grouped[df_grouped.Destino.isin(df_arranjos_filtrado['Cod_Mun_Origem'].unique())]
    df_grouped[['Estudo', 'Trabalho', 'Trabalho e Estudo']] = df_grouped[['Estudo', 'Trabalho', 'Trabalho e Estudo']].astype(np.int64)

    df_grouped['Movimento Pendular'] = df_grouped['Estudo'] + df_grouped['Trabalho'] - df_grouped['Trabalho e Estudo']

    list_dicts.append(df_grouped)

df_movimento_pendular = pd.concat(list_dicts)

df_coordinates = pd.read_csv('https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv')

dict_lat = {row[1]['codigo_ibge']:row[1]['latitude'] for row in df_coordinates.iterrows()}
dict_long = {row[1]['codigo_ibge']:row[1]['longitude'] for row in df_coordinates.iterrows()}

df_movimento_pendular['Latitude Origem'] = df_movimento_pendular.Origem.map(dict_lat)
df_movimento_pendular['Longitude Origem'] = df_movimento_pendular.Origem.map(dict_long)
df_movimento_pendular['Latitude Destino'] = df_movimento_pendular.Destino.map(dict_lat)
df_movimento_pendular['Longitude Destino'] = df_movimento_pendular.Destino.map(dict_long)

df_municipality_name = pd.read_csv('data/territorio/municipios_brasileiros.csv', sep=';')

dict_name = {row[1]['cod']:row[1]['municipio'] for row in df_municipality_name.iterrows()}

df_movimento_pendular['Município Origem'] = df_movimento_pendular.Origem.map(dict_name)
df_movimento_pendular['Município Destino'] = df_movimento_pendular.Destino.map(dict_name)

df_movimento_pendular = df_movimento_pendular[[
    'Arranjo', 'Município Origem', 'Origem','Latitude Origem', 'Longitude Origem'
    , 'Município Destino', 'Destino', 'Latitude Destino', 'Longitude Destino'
    , 'Estudo', 'Trabalho','Trabalho e Estudo', 'Movimento Pendular']]

df_movimento_pendular.rename(columns={'Origem':'CodOrigem','Destino':'CodDestino'}, inplace=True)

In [88]:
df_movimento_pendular.to_csv('data/pop/commuting/commuting.csv', index=False)